In [1]:
from load_utils import prepare_data
from seq2seq_model import *
import numpy as np
import torch
from tqdm.notebook import tnrange, tqdm_notebook

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
q, a, pairs, vector = prepare_data('test', 'glove.42B.300d/glove.42B.300d.txt', small=True)

Reading test -------
Read 4041 sentence pairs
Counting words
Counted words:
In questions: 5087 words
In answers: 394 words


In [4]:
matrix_len = q.n_words
weights_matrix = np.zeros((matrix_len, 300))
word_found = 0
for i, word in enumerate(q.word2index):
    try:
        weights_matrix[i] = vector[word]
    except:
        continue